In [ ]:
#Домашняя работа 6 ФИО:Михайлова Ксения

In [3]:
import pandas as pd
from urllib import parse
import numpy as np

In [74]:
import datetime

In [166]:
from datetime import datetime
from datetime import date
from datetime import timedelta

In [ ]:
#1 гео-классификатор, который каждой строке выставляет географическую принадлежность к определенному региону

In [5]:
keywords = pd.read_csv('keywords.csv')

In [6]:
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [21]:
def geo_locates (row):
     geo_data = {
 
     'Центр': ['москва', 'тула', 'ярославль'],
 
     'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 
     'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
 
     }
     
     for key in geo_data:
            for i in geo_data[key]:
                if row['keyword'] == i:
                    return key
                
    
     return 'undefined'

In [22]:
# Для примера по слову "Москва"
keywords['region'] = keywords.apply(geo_locates, axis=1)
keywords[ (keywords['keyword']=='москва')].head()

,keyword,shows,region
1259,москва,153050,Центр


In [ ]:
#2 Классификация фильмов по рейтингу

In [23]:
ratings = pd.read_csv('ratings.csv')

In [24]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [28]:
group_by_movies = ratings.groupby(by=['movieId']).mean()

In [29]:
group_by_movies.head()

,userId,rating,timestamp
movieId,,,
1,338.558704,3.872470,1.103116e+09
2,318.906542,3.401869,1.069321e+09
3,374.423729,3.161017,9.662429e+08
4,355.538462,2.384615,9.277797e+08
5,320.785714,3.267857,9.967201e+08


In [33]:
def rating_classification (row):
    if row['rating'] <= 2:
        return 'low'
    elif row['rating'] <= 4: 
        return 'medium'
    elif row['rating'] >= 4.5:
        return 'high'

In [34]:
# Результат классификации
group_by_movies['class'] = group_by_movies.apply(rating_classification, axis=1)
movies_class = group_by_movies.filter(items = ['movieId', 'rating', 'class'])
movies_class.head()

,rating,class
movieId,,
1,3.872470,medium
2,3.401869,medium
3,3.161017,medium
4,2.384615,medium
5,3.267857,medium


In [ ]:
#3 Считаем Lifetime - разницу между максимальным и минимальным значением timestamp для каждого пользователя.

In [142]:
ratings2 = pd.read_csv('ratings.csv')

In [143]:
ratings2.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [144]:
filtred_by_user = ratings2.groupby(by=['userId'])['rating'].agg(['count'])

In [145]:
users = filtred_by_user[filtred_by_user['count'] > 100]

In [146]:
merge_dt = pd.merge(users, ratings2, on='userId', how='right')

In [147]:
dates = merge_dt.groupby(by=['userId'])['timestamp'].agg(['min','max'])
dates.head()

,min,max
userId,,
1,1260759108,1260759205
2,835355395,835356246
3,1298861589,1298932787
4,949778714,949982274
5,1163373044,1163375145


In [148]:
def lifetime (row):
    mind = int(row['min'])
    maxd = int(row['max'])
    min_date = datetime.utcfromtimestamp(mind)
    max_date = datetime.utcfromtimestamp(maxd)
    f_date = datetime.date(min_date)
    l_date = datetime.date(max_date)
    lifetime = l_date - f_date
    return lifetime.days

In [149]:
dates['lifetime'] = dates.apply(lifetime, axis=1)

In [157]:
dates_filtred = dates.filter(items = ['userId', 'lifetime'])

In [162]:
average_lifetime = dates_filtred['lifetime'].mean()

In [164]:
print(average_lifetime) #среднее значение Lifetime киноманов

201.09687034277198


In [ ]:
#4 Проверяем верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

In [190]:
import calendar

In [195]:
years = []

In [196]:
def data_range(start_date, end_date): 
    dates = []
    start_date_dt = datetime.strptime(start_date, '%Y')
    end_date_dt = datetime.strptime(end_date, '%Y')
    current_dt = start_date_dt
    years.append(current_dt.year)
    current_dt += timedelta(days=365)
    if calendar.isleap(current_dt.year):
        current_dt += timedelta(days=366)
    else:
        current_dt += timedelta(days=365)
    
    while current_dt < end_date_dt:
        dates.append(current_dt.strftime('%Y'))
        if calendar.isleap(current_dt.year):
             current_dt += timedelta(days=366)
        else:
             current_dt += timedelta(days=365)
        years.append(current_dt.year)
    

In [197]:
data_range('1950', '2010')

In [199]:
movies = pd.read_csv('movies.csv')

In [200]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [213]:
def production_year (row):
        year = '1900'
        for i in years:
            tmp = str(i)
            str_row = str(row['title'])
            find_year = str_row.find(tmp)
            if find_year > 0:
                year = tmp           
        return year     

In [214]:
movies['year'] = movies.apply(production_year, axis=1)

In [215]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [216]:
movies_and_years = movies.filter(items = ['movieId', 'year'])

In [218]:
merge_movies = pd.merge(movies_and_years, ratings2, on='movieId', how='right')

In [219]:
merge_movies.head()

,movieId,year,userId,rating,timestamp
0,1,1995,7,3.0,851866703
1,1,1995,9,4.0,938629179
2,1,1995,13,5.0,1331380058
3,1,1995,15,2.0,997938310
4,1,1995,19,3.0,855190091


In [231]:
#Средний рейтинг фильмов по годам
merge_movies.groupby(by=['year'])['rating'].agg(['mean']).sort_values('mean', ascending=False).head(10)

,mean
year,
1957,4.014241
1972,4.011136
1974,3.999058
1954,3.994220
1962,3.952446
1950,3.915254
1977,3.905786
1964,3.841492
1959,3.841033
